# Translation of an xdf file to snirf format 

We have the same nirs data in two formats: xdf and snirf. 

The xdf format is a general format for storing time series data (https://github.com/sccn/xdf). 

The snirf format is a format for storing nirs data (https://github.com/fNIRS/snirf).



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt

# Flag to run tests and visualizations for each function 
doRunTests = True

# define the files to be used

xdf_fullFile = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/AgePie/015_AgePie_20211112_1_r(1).xdf"
s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/AgePie/AgePie_A16.snirf"

# xdf_fullFile = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/ReArm.lnk/twoTestPatientsForOXY4/C1P07_20210802_1_r.xdf"
# s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/ReArm.lnk/twoTestPatientsForOXY4/C1P07_20210802_1_r.snirf"

## Load the xdf file and return only the NIRS and Event streams

In [ ]:
import pyxdf


def get_NIRS_and_Event_streams(x_file):
    """
    Load the xdf file and returns only the NIRS and Event streams
    """
    # load only the NIRS and Event streams
    data, header = pyxdf.load_xdf(
        filename=x_file,
        select_streams=[{"type": "NIRS"}, {"type": "Event"}],
        synchronize_clocks=True,
        dejitter_timestamps=True,
        verbose=False,
    )
    # find the nirs stream among the list of streams
    for i in range(len(data)):
        if data[i]["info"]["type"][0] == "NIRS":
            nirsStream = data[i]
            break
    # find the Event stream among the list of streams
    for i in range(len(data)):
        if data[i]["info"]["type"][0] == "Event":
            eventStream = data[i]
            break

    return nirsStream, eventStream


if doRunTests:
    # load the xdf file and get the NIRS and Event streams
    nirsStream, eventStream = get_NIRS_and_Event_streams(xdf_fullFile)
    # print the name and type of each retruned stream
    print("File: {}".format(xdf_fullFile))
    print(
        "Nirs : {}, {}".format(
            nirsStream["info"]["name"][0], nirsStream["info"]["type"][0]
        )
    )
    print(
        "Event: {}, {}".format(
            eventStream["info"]["name"][0], eventStream["info"]["type"][0]
        )
    )
    # print the number of samples in each stream
    print("Nirs : {}".format(nirsStream["time_series"].shape))


# Explore the nirs data stream


In [ ]:
if doRunTests:
    nirsData_xdf = nirsStream["time_series"]
    nirsTime_xdf = nirsStream["time_stamps"]
    eventData_xdf = eventStream["time_series"]
    eventTime_xdf = eventStream["time_stamps"]

    # find the set of possible events using np.unique
    events = np.unique(eventData_xdf)
    print("Found {} events with labels in {}".format(len(eventData_xdf), events))

# Reorganize the xdf channels as it is in the corresponding snirf file

In the XDF file, we have 34 channels, but only 16 are of interest, i.e., only channels 0 to 7 and 24 to 31 are effectively used for the nirs data that is also present in the snirf file. It seems that the channels are organized in the following way:
 - channels 0 to 7 are the channels on the left hemisphere
 - channels 24 to 31 are the channels on the right hemisphere

However, in the snirf file, the channels are organized in the following way:
- channels with the lowest wavelength first (i.e., 757 nm)
- channels with the highest wavelength last (i.e., 852 nm)

Moreover, the channels values are stored as a log of the inverse of the intensity.
# modify the data according to the ARTINIS matlab code

In the ARTINIS matlab code, the data is transformed as follows:
```matlab
    data.dataTimeSeries = 1./exp(log(10).* [rawvals(:, 2:2:end) rawvals(:, 1:2:end)]); %change dataTimeSeries to correct values
```
 In python, we can do the same thing with the following code:
```python
    data.dataTimeSeries = 1./np.exp(np.log(10)*np.concatenate((rawvals[:, 1::2], rawvals[:, 0::2]), axis=1))
```

The key question is why do we need to transform the values to the log of the inverse of the intensity?

```python
    data.x = 1./np.exp(np.log(10)* x) 
    # which is equivalent to
    data.x = (1.0 / 10.0) ** x
```

By definition, optical density is $ OD = log_{10}(\frac{I_0}{I}) $, where $I_0$ is the incident light intensity and $I$ is the transmitted light intensity, 

As snirf stores the data in the form of optical density, it comes that xdf data is the base 10 logarithm of the inverse of the optical density:  
$$ y = [\frac{1}{10}]^x \Leftrightarrow \frac{1}{y} = 10^x \Leftrightarrow  log_{10}(\frac{1}{y}) = x $$
where $y$ is the optical density and $x$ is the xdf data.



In [ ]:
def print_xdf_stream_labels(stream):
    """
    Print the labels of the channels by channel number
    """

    channels = []
    for chan in stream["info"]["desc"][0]["channels"][0]["channel"]:
        label = chan["label"]
        unit = chan["unit"]
        type = chan["type"]
        channels.append({"label": label, "unit": unit, "type": type})
    print("Found {} channels: ".format(len(channels)))
    for i in range(len(channels)):
        print(
            "  {:02d}: {} ({} {})".format(
                i,
                channels[i]["label"][0][8:],  # remove the first 8 characters
                channels[i]["type"][0],
                channels[i]["unit"][0],
            )
        )


def print_xdf_stream_labels_and_first_last_data(stream):
    """
    Print the labels of the channels + first data value by channel number
    """
    channels = []
    for chan in stream["info"]["desc"][0]["channels"][0]["channel"]:
        label = chan["label"]
        unit = chan["unit"]
        type = chan["type"]
        channels.append({"label": label, "unit": unit, "type": type})
    print("Found {} channels: ".format(len(channels)))
    for i in range(len(channels)):
        print(
            "  {:02d}: {} ({} {}) [{:5.3f}...{:5.3f}]".format(
                i,
                channels[i]["label"][0][8:],  # remove the first 8 characters
                channels[i]["type"][0],
                channels[i]["unit"][0],
                stream["time_series"][0, i],
                stream["time_series"][-1, i],
            )
        )


def xdf_reorganize_channels_as_in_snirf(nirsStream):
    """
    Reorganize the xdf stream channels as it is in the snirf file
    """
    # if the stream already has 16 channels, do nothing
    if len(nirsStream["info"]["desc"][0]["channels"][0]["channel"]) == 16:
        return nirsStream

    # # modify the data according to the ARTINIS matlab code
    # # data.dataTimeSeries = 1./exp(log(10).* [rawvals(:, 2:2:end) rawvals(:, 1:2:end)]);%change dataTimeSeries to correct values
    # In the XDF file, we have 34 channels, but only 16 are of interest
    # only channels 0 to 7 and 24 to 31 are effectively used 
    # and the order should be changed to match the snirf file (small wavelength first)

    new_channel_order = [
        1,
        3,
        5,
        7,
        25,
        27,
        29,
        31,
        0,
        2,
        4,
        6,
        24,
        26,
        28,
        30,
    ]

    # keep only the 16 channels used and in the snirf order
    # do the same for the time series and the channel labels
    channels = []
    time_series = np.zeros((len(nirsStream["time_series"]), len(new_channel_order)))
    for i in range(len(new_channel_order)):
        iNew = new_channel_order[i]
        channels.append(nirsStream["info"]["desc"][0]["channels"][0]["channel"][iNew])
        time_series[:, i] = nirsStream["time_series"][:, iNew]

    # modify the stream itself
    nirsStream["info"]["desc"][0]["channels"][0]["channel"] = channels
    nirsStream["time_series"] = time_series

    # convert the modified stream to the correct values for snirf
    # NOTE: comment out => only change the order of the channels (for verification)
    # NOTE: the two following lines are equivalent to the matlab code above
    #nirsStream["time_series"] = 1.0 / 10.0 ** nirsStream["time_series"] 
    nirsStream["time_series"] = 1.0 / np.exp(np.log(10) * nirsStream["time_series"])

    return nirsStream


if doRunTests:
    nirsStream = xdf_reorganize_channels_as_in_snirf(nirsStream)


# Copy the original template snirf file to a new file

In [ ]:
from snirf import Snirf
import os

def copy_snirf_file(file_name, new_file_name):
    snirf = Snirf(file_name, "r")
    snirf.save(new_file_name)
    snirf.close()  

if doRunTests:
    template_file = "new_AgePie_A16_2.snirf"

    # get the file name without the path and extension from the xdf file
    file_name = os.path.basename(xdf_fullFile)
    new_fname = os.path.splitext(file_name)[0]+".snirf"
    copy_snirf_file(template_file, new_fname)

In [ ]:
import numpy as np

snirf = Snirf(new_fname, "r+")

a = 1
snirf.close()


In [ ]:


def get_events_100_111(event_data):
    """
    Find all events containing the word 111 and 100
    """
    i111 = []
    i100 = []
    for i in range(len(event_data)):
        if "111" in event_data[i][0]:
            i111.append(i)
        if "100" in event_data[i][0]:
            i100.append(i)
    print("Found {} events 111".format(len(i111)))
    print("Found {} events 100".format(len(i100)))
    return i111, i100
 

event_data = eventStream["time_series"]
event_time = eventStream["time_stamps"]

i111, i100 = get_events_100_111(event_data)

for i in i111:
    print("Event {}: {} at {}".format(i, event_data[i][0][3:], event_time[i]))

for i in i100:
    print("Event {}: {} at {}".format(i, event_data[i][0][3:], event_time[i]))

data = []
for i in i111:
    data.append([event_time[i], 5.0, 1.0])




# Prepare the data for the snirf file

In [ ]:
nirs_data = nirsStream["time_series"]
nirs_time = nirsStream["time_stamps"]


event_data = eventStream["time_series"]
event_time = eventStream["time_stamps"]


# # modify the annotations 
# beg = snirf.nirs[0].data[0].time[delay]
# end = snirf.nirs[0].data[0].time[delay + duration]
# print("beg = {}, end = {}, duration ={}".format(beg, end, end - beg))

# annotations = snirf.nirs[0].stim
# n_annotations = len(annotations)
# # the first time is always ZERO, hence the annotations are shifted by beg
# for a in range(n_annotations):
#     if annotations[a].data.ndim == 1:
#         annotations[a].data[0] = annotations[a].data[0] - beg
#     else:
#         for i in range(len(annotations[a].data)):
#             annotations[a].data[i][0] = annotations[a].data[i][0] - beg
